In [ ]:
import cv2
import numpy as np

# Load images
apple = cv2.imread('apple.jpg')
orange = cv2.imread('orange.jpg')

print("Apple shape:", apple.shape)
print("Orange shape:", orange.shape)

# Combine halves of apple and orange directly for comparison
apple_orange = np.hstack((apple[:, :256], orange[:, 256:]))

# Generate Gaussian pyramids
def generate_gaussian_pyramid(image, levels=2):
    gp = [image.copy()]
    for _ in range(levels):
        image = cv2.pyrDown(image)
        gp.append(image)
    return gp

gp_apple = generate_gaussian_pyramid(apple)
gp_orange = generate_gaussian_pyramid(orange)

# Generate Laplacian pyramids
def generate_laplacian_pyramid(gaussian_pyramid):
    lp = [gaussian_pyramid[-1]]
    for i in range(len(gaussian_pyramid) - 1, 0, -1):
        gaussian_expanded = cv2.pyrUp(gaussian_pyramid[i])
        size = (gaussian_pyramid[i-1].shape[1], gaussian_pyramid[i-1].shape[0])
        gaussian_expanded = cv2.resize(gaussian_expanded, size)
        laplacian = cv2.subtract(gaussian_pyramid[i-1], gaussian_expanded)
        lp.append(laplacian)
    return lp

lp_apple = generate_laplacian_pyramid(gp_apple)
lp_orange = generate_laplacian_pyramid(gp_orange)

# Blend images at each pyramid level
apple_orange_pyramid = []
for apple_lap, orange_lap in zip(lp_apple, lp_orange):
    cols, rows, _ = apple_lap.shape
    laplacian = np.hstack((apple_lap[:, :cols // 2], orange_lap[:, cols // 2:]))
    apple_orange_pyramid.append(laplacian)

# Reconstruct the blended image
apple_orange_reconstruct = apple_orange_pyramid[0]
for i in range(1, len(apple_orange_pyramid)):
    apple_orange_reconstruct = cv2.pyrUp(apple_orange_reconstruct)
    size = (apple_orange_pyramid[i].shape[1], apple_orange_pyramid[i].shape[0])
    apple_orange_reconstruct = cv2.resize(apple_orange_reconstruct, size)
    apple_orange_reconstruct = cv2.add(apple_orange_pyramid[i], apple_orange_reconstruct)

# Display results
cv2.imshow("Apple", apple)
cv2.imshow("Orange", orange)
cv2.imshow("Direct Blend (Half Apple + Half Orange)", apple_orange)
cv2.imshow("Blended Pyramid Image", apple_orange_reconstruct)

cv2.waitKey(0)
cv2.destroyAllWindows()


Apple shape: (512, 512, 3)
Orange shape: (512, 512, 3)
